# OOPython

Задача 4. Функциональные пространства.

Борисов Дмитрий, 373гр.

### Воспользуемся ранее получеными результатами.
#### 1. Иерархия классов для формул численного дифференцирования:
* Forward 1st Order Derivative - ForwardDerivative
* Backward 1st Order Derivative - BackwardDerivative
* Central 2nd Order Derivative - CentralDerivative
* Symmetric 4th Order Derivative - FourthDerivative
* Symmetric 6th Order Derivative - FifthDerivative

In [1]:
class AbstractDerivative:
    def __init__(self, function, step = 1e-5):
        self._function = function
        self._step = float(step)
        
    def __call__(self, x):
        raise NotImplementedError
        
class ForwardDerivative(AbstractDerivative):
    __MethodName = 'Forward 1st Order Derivative'
    def getName(self):
        return self.__MethodName

    def __call__(self, x):
        return (self._function(x + self._step) - self._function(x)) / self._step

class BackwardDerivative(AbstractDerivative):
    __MethodName = 'Backward 1st Order Derivative'
    def getName(self):
        return self.__MethodName
 
    def __call__(self, x):
        return (self._function(x) - self._function(x - self._step)) / self._step

class CentralDerivative(AbstractDerivative):
    __MethodName = 'Central 2nd Order Derivative'
    def getName(self):
        return self.__MethodName

    def __call__(self, x):
        return (self._function(x + self._step) - self._function(x - self._step)) / (2 * self._step)
    
class FourthDerivative(AbstractDerivative):
    __MethodName = 'Symmetric 4th Order Derivative'
    def getName(self):
        return self.__MethodName

    def __call__(self, x):
        return 4. / 3 * (self._function(x + self._step) - self._function(x - self._step)) / (2 * self._step) \
    - 1. / 3 * (self._function(x + 2 * self._step) - self._function(x - 2 * self._step)) / (4 * self._step)
    
class FifthDerivative(AbstractDerivative):
    __MethodName = 'Symmetric 6th Order Derivative'
    def getName(self):
        return self.__MethodName

    def __call__(self, x):
        return 3. / 2 * (self._function(x + self._step) - self._function(x - self._step)) / (2 * self._step) \
    - 3. / 5 * (self._function(x + 2 * self._step) - self._function(x - 2 * self._step)) / (4 * self._step) \
    + 1. / 10 * (self._function(x + 3 * self._step) - self._function(x - 3 * self._step)) / (6 * self._step)

#### 2. Иерархия классов для формул численного интегрирования:
* метод левых прямоугольников - LeftRectangleIntegral
* метод правых прямоугольников - RightRectangleIntegral
* метод средних прямоугольников - MiddleRectangleIntegral
* метод трапеций - TrapeziumIntegral
* метод Симпсона 4-гопорядка точности - SimpsonIntegral

In [2]:
import numpy as np
class AbstractIntegral:
    _MethodName = 'Abstract Integral'
    _shift = 0.
        
    def ChooseFunction(self, function):
        self._function = function
    
    def SetLeftBorder(self, leftBorder):
        self._leftBorder = leftBorder
        
    def SetGrid(self, step):
        self._step = step
        
    def SetCoeffients(self, points):
        raise NotImplementedError
    
    def getName(self):
        return self._MethodName
    
    def __call__(self, x):
        self._gridArray = np.arange(self._leftBorder, x + self._step, self._step)   
        self._gridLength = len(self._gridArray)
        self.SetCoeffients(self._gridLength)
        
        f, left, right, coeff = self._function, self._leftBorder, x, self._coeffients    
        
        value = 0.
        for i in range(self._gridLength):
            xCurrent = self._gridArray[i]
            value += coeff[i] * f(xCurrent + self._shift * self._step)
            
        value *= self._step
        return value     
        
class LeftRectangleIntegral(AbstractIntegral):
    _MethodName = 'Left Rectangle Integral'
    
    def SetCoeffients(self, points):
        self._coeffients = [1] * (points - 1) + [0]

class RightRectangleIntegral(AbstractIntegral):
    _MethodName = 'Right Rectangle Integral'
    
    def SetCoeffients(self, points):
        self._coeffients = [0] + [1] * (points - 1)

class MiddleRectangleIntegral(AbstractIntegral):
    _MethodName = 'Middle Rectangle Integral'
    _shift = 0.5
    
    def SetCoeffients(self, points):
        self._coeffients = [1] * (points - 1) + [0.]

class TrapeziumIntegral(AbstractIntegral):
    _MethodName = 'Trapezium Integral'
    
    def SetCoeffients(self, points):
        self._coeffients = [0.5] + [1] * (points - 2) + [0.5]
    
class SimpsonIntegral(AbstractIntegral):
    _MethodName = 'Simpson Integral'
    
    def SetCoeffients(self, points):
        self._coeffients = [1./3] + [4./3, 2./3] * ((points - 2) / 2)
        if (points % 2 == 1):
            self._coeffients += [4./3]
        self._coeffients += [1./3]

### Часть 1: классы, реализующие 4 различных нормированных-метрических пространства

In [3]:
import numpy as np
import sympy as smp
xSym = smp.Symbol('x')

class AbstractSpace:
    def __init__(self, a, b):
        self._leftBorder = a
        self._rightBorder = b
    
    def SetGrid(self, step = 1e-3):
        self._step = step  
        
    def MaxAbsoluteValueOfFunction(self, function):
        self._gridArray = np.arange(self._leftBorder, self._rightBorder + self._step, self._step)            
        maxValue, value = 0., 0.
        
        for x in self._gridArray:
            value = abs(function(x))
            if value > maxValue:
                maxValue = value
        return maxValue
  
    def MaxAbsoluteValueOfFirstFunctionDerivative(self, function):
        self._gridArray = np.arange(self._leftBorder, self._rightBorder + self._step, self._step)
        maxValue, value = 0., 0.
        
        ListOfNumericDerivates = (CentralDerivative(function, self._step), \
                                  ForwardDerivative(function, self._step), \
                                  BackwardDerivative(function, self._step))

        for x in self._gridArray:
            if x > self._leftBorder and x < self._leftBorder:
                value = abs(ListOfNumericDerivates[0](x))
            elif x == self._leftBorder:
                value = abs(ListOfNumericDerivates[1](x))
            elif x == self._rightBorder:
                value = abs(ListOfNumericDerivates[2](x))
     
            if value > maxValue:
                maxValue = value
        return maxValue
   
    def MaxAbsoluteValueOfSecondFunctionDerivative(self, function):
        self._gridArray = np.arange(self._leftBorder, self._rightBorder + self._step, self._step)
        maxValue, value = 0., 0.
        
        ListOfNumericDerivates = (ForwardDerivative(function, self._step), \
                                  BackwardDerivative(function, self._step))
        
        for x in self._gridArray:
            if x > self._leftBorder and x < self._leftBorder:
                #2nd Order Second Derivative, 3-point stencil(x-h, x, x+h)
                value = (ListOfNumericDerivates[0](x) - ListOfNumericDerivates[0](x - self._step)) / self._step
            
            elif x == self._leftBorder:
                #1st Order Second Derivative, 3-point stencil(x, x+h, x+2h)
                value = (ListOfNumericDerivates[0](x + self._step) - ListOfNumericDerivates[0](x)) / self._step
            
            elif x == self._rightBorder:
                #1st Order Second Derivative, 3-point stencil(x-2h, x-h, x)
                value = (ListOfNumericDerivates[1](x) - ListOfNumericDerivates[1](x - self._step)) / self._step
     
            value = abs(value)
            if value > maxValue:
                maxValue = value
        return maxValue
        
    def CalculateNorm(self, function):
        raise NotImplementedError

    def CalculateMetric(self, function1, function2):
        return self.CalculateNorm(function1 - function2)

class SpaceC(AbstractSpace):
    def CalculateNorm(self, function):
        function = smp.lambdify(xSym, function)
        return self.MaxAbsoluteValueOfFunction(function)

class SpaceC1(AbstractSpace):
    def CalculateNorm(self, function):
        function = smp.lambdify(xSym, function)
        return (self.MaxAbsoluteValueOfFunction(function) + \
                self.MaxAbsoluteValueOfFirstFunctionDerivative(function))

class SpaceC2(AbstractSpace):
    def CalculateNorm(self, function):
        function = smp.lambdify(xSym, function)
        return (self.MaxAbsoluteValueOfFunction(function) + \
                self.MaxAbsoluteValueOfFirstFunctionDerivative(function) + \
                self.MaxAbsoluteValueOfSecondFunctionDerivative(function))

class SpaceL2(AbstractSpace):
    def CalculateNorm(self, function):
        function = smp.lambdify(xSym, function**2)
        integral = TrapeziumIntegral()
        integral.ChooseFunction(function)
        integral.SetGrid(self._step)
        integral.SetLeftBorder(self._leftBorder)
        value = (integral(self._rightBorder))**(0.5)
        return value

### Часть 2:
* класс, реализующий 1 предгильбертово пространство со стандартным правилом вычисления скалярного произведения
* класс, реализующий 1 нормированное пространство с нормой, порождаемой скалярным произведением предгильбертова пространства E
* класс, реализующий 1 метрическое пространство с метрикой, порождаемой нормой нормированного пространства

In [4]:
class SpacePreHilbert(AbstractSpace):
    def ScalarMultiplication(self, function1, function2):
        function = smp.lambdify(xSym, function1 * function2)
        integral = TrapeziumIntegral()
        integral.ChooseFunction(function)
        integral.SetGrid(self._step)
        integral.SetLeftBorder(self._leftBorder)
        value = integral(self._rightBorder)
        return value
    
    def CalculateNorm(self, function):
        value = (self.ScalarMultiplication(function, function))**(0.5)
        return value
    
    #CalculateMetric(self, function1, function2) уже реализовано в AbstractSpace

### Тестирование на функциях

In [5]:
listOfSymbolicFunctions = [5./(2 + 3 * xSym**2), 2./ (5 + smp.cos(xSym)), (3 + 4 * xSym**2)**(1./3),\
                           2. * smp.exp(-xSym**2) / smp.pi**(0.5)]

NameOfFunctions = ['5/(2 + 3 * x**2)', '2/(5 + cos(x))', '(3 + 4 * x**2)**(1/3)',\
                   '2 * exp(-x**2) / pi**(1/2)']

* В 4-х нормированных пространствах C0[0; 2], С1[0; 2], C2[0; 2], L2[0; 2] вычислить нормы всех функций

In [6]:
leftBorder, rightBorder, h = 0., 2., 1e-3

listOfSpaces = [SpaceC(leftBorder, rightBorder), SpaceC1(leftBorder, rightBorder), \
                SpaceC2(leftBorder, rightBorder), SpaceL2(leftBorder, rightBorder)]

for space in listOfSpaces:
    space.SetGrid(h)

NameOfSpaces = ['C0[0; 2]', 'C1[0; 2]', 'C2[0; 2]', 'L2[0; 2]']

In [7]:
import pandas as pd
from IPython.display import display

data = []
for function in listOfSymbolicFunctions:
    dataIn = []
    for space in listOfSpaces:
        dataIn.append(space.CalculateNorm(function))  
    data.append(dataIn)                
        
df = pd.DataFrame(data = data, index = NameOfFunctions, columns = NameOfSpaces)
print('Нормы функций в соответствующих нормированных пространствах:')
display(df)

Нормы функций в соответствующих нормированных пространствах:


,C0[0; 2],C1[0; 2],C2[0; 2],L2[0; 2]
5/(2 + 3 * x**2),2.500000,2.806308,10.306230,1.977840
2/(5 + cos(x)),0.436314,0.522868,0.578424,0.524045
(3 + 4 * x**2)**(1/3),2.668402,3.417450,4.699445,2.815513
2 * exp(-x**2) / pi**(1/2),1.128379,1.211192,3.467942,0.893216


* В 4-х реализованных соответствующих метрических пространствах вычислить попарные расстояния между всеми функциями

In [8]:
for space, name in zip(listOfSpaces, NameOfSpaces):
    data = []   
    for function1 in listOfSymbolicFunctions:
        dataIn = []
        for function2 in listOfSymbolicFunctions:
            dataIn.append(space.CalculateMetric(function1, function2))         
        data.append(dataIn)
     
    print('Distance between functions in space ' + name)
    df = pd.DataFrame(data = data, index = NameOfFunctions, columns = NameOfFunctions)
    display(df)

Distance between functions in space C0[0; 2]


,5/(2 + 3 * x**2),2/(5 + cos(x)),(3 + 4 * x**2)**(1/3),2 * exp(-x**2) / pi**(1/2)
5/(2 + 3 * x**2),0.000000,2.166667,2.311259,1.371621
2/(5 + cos(x)),2.166667,0.000000,2.232087,0.795046
(3 + 4 * x**2)**(1/3),2.311259,2.232087,0.000000,2.647735
2 * exp(-x**2) / pi**(1/2),1.371621,0.795046,2.647735,0.000000


Distance between functions in space C1[0; 2]


,5/(2 + 3 * x**2),2/(5 + cos(x)),(3 + 4 * x**2)**(1/3),2 * exp(-x**2) / pi**(1/2)
5/(2 + 3 * x**2),0.000000,2.559529,3.366615,1.595116
2/(5 + cos(x)),2.559529,0.000000,2.894581,0.964413
(3 + 4 * x**2)**(1/3),3.366615,2.894581,0.000000,3.479595
2 * exp(-x**2) / pi**(1/2),1.595116,0.964413,3.479595,0.000000


Distance between functions in space C2[0; 2]


,5/(2 + 3 * x**2),2/(5 + cos(x)),(3 + 4 * x**2)**(1/3),2 * exp(-x**2) / pi**(1/2)
5/(2 + 3 * x**2),0.000000,10.115006,12.148532,6.838287
2/(5 + cos(x)),10.115006,0.000000,4.121021,3.276719
(3 + 4 * x**2)**(1/3),12.148532,4.121021,0.000000,7.018341
2 * exp(-x**2) / pi**(1/2),6.838287,3.276719,7.018341,0.000000


Distance between functions in space L2[0; 2]


,5/(2 + 3 * x**2),2/(5 + cos(x)),(3 + 4 * x**2)**(1/3),2 * exp(-x**2) / pi**(1/2)
5/(2 + 3 * x**2),0.000000,1.575669,1.851987,1.101938
2/(5 + cos(x)),1.575669,0.000000,2.295393,0.618643
(3 + 4 * x**2)**(1/3),1.851987,2.295393,0.000000,2.326812
2 * exp(-x**2) / pi**(1/2),1.101938,0.618643,2.326812,0.000000


* В реализованном предгильбертовом пространстве вычислить попарные скалярные произведения между всеми функциями

In [9]:
space = SpacePreHilbert(leftBorder, rightBorder)
space.SetGrid(h)
data = []
    
for function1 in listOfSymbolicFunctions:
    dataIn = []
    for function2 in listOfSymbolicFunctions:
        dataIn.append(space.ScalarMultiplication(function1, function2))         
    data.append(dataIn)
     
print('Scalar Multiplication between functions in Pre-Hilbert space')
df = pd.DataFrame(data = data, index = NameOfFunctions, columns = NameOfFunctions)
display(df)

Scalar Multiplication between functions in Pre-Hilbert space


,5/(2 + 3 * x**2),2/(5 + cos(x)),(3 + 4 * x**2)**(1/3),2 * exp(-x**2) / pi**(1/2)
5/(2 + 3 * x**2),3.911852,0.851871,4.204555,1.747709
2/(5 + cos(x)),0.851871,0.274623,1.466455,0.344869
(3 + 4 * x**2)**(1/3),4.204555,1.466455,7.927115,1.655447
2 * exp(-x**2) / pi**(1/2),1.747709,0.344869,1.655447,0.797834


* В реализованном предгильбертовом пространстве попарно вычислить углы между всеми функциями

In [10]:
data = []
    
for function1 in listOfSymbolicFunctions:
    dataIn = []
    for function2 in listOfSymbolicFunctions:
        value = space.ScalarMultiplication(function1, function2) / \
                (space.CalculateNorm(function1) * space.CalculateNorm(function2))
        dataIn.append(value)         
    data.append(dataIn)
     
print('Cosines of angles between functions in Pre-Hilbert space')
df = pd.DataFrame(data = data, index = NameOfFunctions, columns = NameOfFunctions)
display(df)

Cosines of angles between functions in Pre-Hilbert space


,5/(2 + 3 * x**2),2/(5 + cos(x)),(3 + 4 * x**2)**(1/3),2 * exp(-x**2) / pi**(1/2)
5/(2 + 3 * x**2),1.000000,0.821891,0.755042,0.989286
2/(5 + cos(x)),0.821891,1.000000,0.993900,0.736765
(3 + 4 * x**2)**(1/3),0.755042,0.993900,1.000000,0.658266
2 * exp(-x**2) / pi**(1/2),0.989286,0.736765,0.658266,1.000000
